In [114]:
import openai
import langchain
import pinecone
from pinecone import Pinecone, ServerlessSpec, CloudProvider, AwsRegion
from langchain_pinecone.vectorstores import PineconeVectorStore
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain_core.documents import Document 


In [14]:
import os


In [97]:
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [96]:
doc=read_doc('documents/')
doc

[Document(metadata={'producer': 'Matplotlib pdf backend v3.6.3', 'creator': 'Matplotlib v3.6.3, https://matplotlib.org', 'creationdate': '2025-11-04T17:14:58+00:00', 'source': 'documents\\rag_demo_data.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='Acme Corp  Internal Knowledge Base (Demo)\nThis PDF is a synthetic demo document you can use to test PDF loaders, embeddings, and\nretrieval. The content is intentionally varied across sections so chunking and splitting\nstrategies have something to work with.  Sections included: 1) Product Overview 2) Pricing\n& Plans 3) Setup & Onboarding 4) Troubleshooting FAQ 5) Release Notes (abridged) 6)\nContact Directory  Tip: This text is purposely a mix of short and long sentences to test\nRecursiveCharacterT extSplitter behavior with different chunk sizes and overlaps.'),
 Document(metadata={'producer': 'Matplotlib pdf backend v3.6.3', 'creator': 'Matplotlib v3.6.3, https://matplotlib.org', 'creationdate': '2025-11-04T17:14:5

In [95]:
#divide the docs into chunks 
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    docs=text_splitter.split_documents(docs)
    return (docs)

In [94]:
documents=chunk_data(docs=doc)
documents

[Document(metadata={'producer': 'Matplotlib pdf backend v3.6.3', 'creator': 'Matplotlib v3.6.3, https://matplotlib.org', 'creationdate': '2025-11-04T17:14:58+00:00', 'source': 'documents\\rag_demo_data.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='Acme Corp  Internal Knowledge Base (Demo)\nThis PDF is a synthetic demo document you can use to test PDF loaders, embeddings, and\nretrieval. The content is intentionally varied across sections so chunking and splitting\nstrategies have something to work with.  Sections included: 1) Product Overview 2) Pricing\n& Plans 3) Setup & Onboarding 4) Troubleshooting FAQ 5) Release Notes (abridged) 6)\nContact Directory  Tip: This text is purposely a mix of short and long sentences to test\nRecursiveCharacterT extSplitter behavior with different chunk sizes and overlaps.'),
 Document(metadata={'producer': 'Matplotlib pdf backend v3.6.3', 'creator': 'Matplotlib v3.6.3, https://matplotlib.org', 'creationdate': '2025-11-04T17:14:5

In [93]:
embeddings = OpenAIEmbeddings(
    api_key=os.environ["OPENROUTER_API_KEY"],
    model="text-embedding-3-large",   # or any supported model
    base_url="https://openrouter.ai/api/v1"
)

In [92]:
vectors = embeddings.embed_query("how are you?")
len(vectors)

3072

In [132]:
os.environ["PINECONE_API_KEY"] = "pcsk_5CX4Px_31FNGf7oNNNcpn2qVuXJjQXAUjSvvtJAeD4cHNC2hBKVJXE84Qj1bEYJTiAfoyq"

pc = Pinecone(api_key="pcsk_5CX4Px_31FNGf7oNNNcpn2qVuXJjQXAUjSvvtJAeD4cHNC2hBKVJXE84Qj1bEYJTiAfoyq")
index_name = pc.Index("chatbox")

In [136]:

vectorstore = PineconeVectorStore.from_documents(
    doc,
    embedding=embeddings,
    index_name=index_name,   # ✅ must be a string
    pinecone_api_key=os.environ["PINECONE_API_KEY"]
)


ValueError: Index '<pinecone.db_data.index.Index object at 0x000001B28505DF20>' not found in your Pinecone project. Did you mean one of the following indexes: chatbox